In [2]:
import pandas as pd
import spacy
import re

# from gensim import word2vec

In [9]:
df = pd.read_csv('./simpsons_dataset.csv')
df = df.dropna().reset_index(drop=True)
df.head(5)

In [23]:

import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Предположим, что у вас есть данные: texts - список текстов, содержащих слова, и numerical_features - массив числовых признаков
numerical_features = list(range(len(df)))

# Создаем модель Word2Vec на основе текстов
word2vec_model = Word2Vec(df.spoken_words, vector_size=100, window=5, min_count=1, workers=4)

# Создаем функцию для преобразования каждого текста в его векторное представление с использованием Word2Vec
def text_to_vector(text):
    words = text.split()
    vector = np.zeros(word2vec_model.vector_size)
    for word in words:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
    return vector / len(words)

# Преобразуем каждый текст в его векторное представление с использованием Word2Vec
text_vectors = np.array([text_to_vector(text) for text in df.spoken_words])



y = np.array([1 if i % 2 == 0 else 0 for i in range(len(df))])
numerical_features = np.array(numerical_features).reshape(-1, 1)

# Объединяем векторные представления текстов с числовыми признаками
X = pd.concatenate((text_vectors, numerical_features), axis=1)

# Предположим, у вас есть метки классов в переменной y

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучаем модель классификации (например, логистическую регрессию)
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказываем метки классов для тестового набора
y_pred = model.predict(X_test)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4984831631105269
